In [1]:
from trainer import Trainer

In [10]:
import torch


def test_model():
    model = torch.nn.Sequential(
        torch.nn.Linear(20, 100), torch.nn.Linear(100, 10), torch.nn.Linear(10, 5)
    )
    return model


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, size):
        self.size = size
        self.data = [(torch.rand(20), torch.rand(1)) for _ in range(size)]

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        return self.data[index]


def prepare_dataloader(
    dataset: torch.utils.data.Dataset, batch_size: int, num_workers_: int = 0
):
    """
    Prepare a DataLoader with distributed sampling.

    Parameters
    ----------
    dataset : Dataset
        The dataset to load.

    batch_size : int
        Number of samples per batch.

    num_workers_ : int, optional
        Number of workers for data loading, by default 1.

    Returns
    -------
    DataLoader
        A DataLoader instance for the specified dataset.

    Notes
    Utilize distributed sampling for better training efficiency.
    """
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=False,
        num_workers=num_workers_,
    )

In [11]:
model = test_model()
dataset = TestDataset(10)
data = prepare_dataloader(dataset, 10)
optim_test = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
trainer_ = Trainer(
    model,
    train_data=data,
    optimizer=optim_test,
    loss_func=criterion,
    resume_checkpoint=False,
)

In [12]:
trainer_.train(max_epochs=4, save_every=2)

2024-01-25 15:16:46.709 | INFO     | trainer:_run_epoch:114 - [GPUcpu] Epoch 0 | Batchsize: 10 | Steps: 1


RuntimeError: 0D or 1D target tensor expected, multi-target not supported